<a href="https://colab.research.google.com/github/HisameOgasahara/HF_transformers/blob/main/HFHT_7_2_Lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn==1.4.2 \
     transformers==4.43.0 \
     datasets==2.20.0 \
     peft==0.10.0 \
     accelerate==0.32.1 \
     bitsandbytes==0.43.1 \
     trl==0.9.6 # Transformer Reinforcement Learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 891.1 kB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3

# 7.2.1 LoRA 및 파생 방법론

### LoRA 초기화 방법론

##### 마치 원래 pathway에 추가 pathway를 주는 skip connection을 떠올려 보자. 단, x가 아니라 ABx가 되게 한 것이고, AB는 학습해야할 파라미터며 원본 웨이트에 AB가 추가분으로 작용. 단 이 connecion은 언제든 원래 모델에 넣었다 뺐다 가능하다는 거.

In [ ]:
from peft import LoraConfig

config = LoraConfig(init_lora_weights="gaussian")

LoftQ

In [ ]:
### 설정만 하지 실제 모델 불러오고 하는 건 아직 아니다.
## 로라는 가중치를 불러오는 게 아니라, 태스크/타겟모듈 별로 설정만 한다.

from peft import replace_lora_weights_loftq, LoraConfig, get_peft_model # LoftQ (Low-rank Factorized Quantization)를 사용하여 LoRA 가중치를 양자화하는 함수, get_peft_model: 기존 모델에 LoRA 어댑터를 추가하여 PEFT 모델을 생성
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4" #4bit 양자화 유형으로 "nf4" (Normal Float 4)를 사용하도록 설정합니다. nf4는 4bit 양자화에서 좋은 성능을 보이는 유형 중 하나
)
model = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    quantization_config=bnb_config #nb_config에서 정의한 양자화 설정을 모델 로드 시에 적용합니다. 이로써 모델은 4bit로 양자화된 상태로 GPU 메모리에 올라가게 됩니다.
)
lora_config = LoraConfig(
    task_type="CAUSAL_LM", #현재 작업이 causal language modeling (텍스트 생성)임을 명시
    target_modules="all-linear" #LoRA를 적용할 대상 모듈을 "all-linear"로 설정합니다. 이는 모델 내의 모든 선형 레이어 (linear layer)에 LoRA 어댑터를 추가하겠다는 의미. 대부분의 파라미터가 선형 레이어에 존재하기 때문에 "all-linear"는 일반적으로 효과적인 설정
)
peft_model = get_peft_model(model, lora_config) #기존의 양자화된 모델 (model)에 lora_config에서 정의한 LoRA 설정을 적용하여 peft model 생성. 즉 이때부터 lora의 추가 레이어가 원래 모델과 공존
replace_lora_weights_loftq(peft_model) # PEFT 모델 (peft_model)의 LoRA 가중치를 LoftQ (Low-rank Factorized Quantization)를 사용하여 양자화

## Gemma 모델이 4bit로 양자화되어 로드된 후, LoRA가 추가되고, LoRA가 추가된 PEFT 모델의 LoRA 가중치를 LoftQ로 양자화**하는 거예요.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

RuntimeError: No GPU found. A GPU is needed for quantization.

In [ ]:
#Rank-stabilized LoRA
config = LoraConfig(use_rslora=True)

In [ ]:
#Weight-Decomposed Low-Rank Adaptation (DoRA)
config = LoraConfig(use_dora=True)

In [ ]:
#QLoRA-style training
config = LoraConfig(target_modules="all-linear")

In [ ]:
#Memory efficient Layer Replication with LoRA
config = LoraConfig(layer_replication=[[0,4], [2,5]])

### Save adapters

In [ ]:
from transformers import AutoModelForCausalLM, AutoConfig
from peft import get_peft_model, LoraConfig, TaskType

# 모델 로드
model = AutoModelForCausalLM.from_pretrained("nlpai-lab/ko-gemma-2b-v1")
print(type(model)) # <class 'transformers.models.gemma.modeling_gemma.GemmaForCausalLM'>. peft되기 전의 타입.

# peft
peft_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

# 모델 저장
model.save_pretrained("/content/peft")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

<class 'transformers.models.gemma.modeling_gemma.GemmaForCausalLM'>
trainable params: 921,600 || all params: 2,507,094,016 || trainable%: 0.036759690467068624


In [ ]:
print(type(model)) #로라가 부착된 경우 타입이 peft가 된다.

<class 'peft.peft_model.PeftModelForSeq2SeqLM'>


In [ ]:
!ls /content/peft
!du -sh /content/peft

adapter_config.json  adapter_model.safetensors	README.md
3.6M	/content/peft


### Merge adapters

##### lora와 합병:
###### - Wx + delWx = (W+delW)x 인 걸 이용. 전자는 서로 다른 두 경로(원본과 로라 경로)를 통해 온걸 더한게 후자(즉 합병처럼 입력텐서에 웨이트가 작용)와 같은 효과.

###### - 병합은 말 그대로 로라웨이트를 웨이트에 아예 더한거. 위에 수식을 생각하면 차이가 느껴질 것. 핵심은 분배법칙의 전(로라) 후(병합)에 있다.

In [ ]:
### 참고로 기본 모델의 ko-gemma는 두개 이상 띄우면 ram만으로는 터진다.

from transformers import AutoModelForCausalLM
from peft import PeftModel

model = AutoModelForCausalLM.from_pretrained("nlpai-lab/ko-gemma-2b-v1")
peft_model_id = "/content/drive/MyDrive/HFbookPractice/trained_model/25_02_19/peft"
peft_model = PeftModel.from_pretrained(model, peft_model_id) # 이 단계에서는 아직 LoRA 어댑터가 기본 모델 가중치에 병합되지 않습니다. peft_model은 기본 모델과 어댑터를 별도로 가지고 있으며, 추론 또는 추가 학습 시에 어댑터가 동적으로 적용되는 방식
peft_model.merge_and_unload() #어댑터 병합 및 언로드. peft_model에 부착되어 있던 LoRA 어댑터의 학습된 가중치를 기본 모델의 해당 레이어 가중치에 더합니다. 이 과정은 수학적으로 LoRA의 업데이트 행렬을 원래 모델 가중치에 반영하는 것

#병합 작업이 완료된 후, 더 이상 필요 없어진 LoRA 어댑터 관련 정보 (어댑터 레이어, 설정 등)를 peft_model 객체에서 제거 (언로드) 합니다. 결과적으로 peft_model은 더 이상 PEFT 모델이 아니라, LoRA가 병합된 일반적인 Transformers 모델

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaR

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained("nlpai-lab/ko-gemma-2b-v1")
peft_model_id = "/content/drive/MyDrive/HFbookPractice/trained_model/25_02_19/peft"
model = PeftModel.from_pretrained(base_model, peft_model_id)
model.merge_adapter()

model.unmerge_adapter()

In [ ]:
## 여러 개의 LoRA (Low-Rank Adaptation) 어댑터를 로드하고, 이들을 가중 평균하여 하나의 새로운 어댑터로 조합

from transformers import AutoModelForCausalLM
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    "nlpai-lab/ko-gemma-2b-v1",
    torch_dtype="float16",
    device_map="auto", # (GPU 우선, 없으면 CPU)
)
peft_model_id = "/content/peft-a"
model = PeftModel.from_pretrained(base_model, peft_model_id, adapter_name="a") #최신 peft에선 이 함수를 통해 이름을 인수로 지정하는 것 만으로도 내부적으로 set_adapter를 호출하는 것과 같은 효과가 나타난다.

weighted_adapter_name = "a-b" #가중 평균될 새로운 어댑터의 이름을 "a-b" 로 지정합니다(괜히 -라 해서 그런데 의미는 +다. 이름에 속지말자). (아직 생성되지는 않았습니다.)
model.load_adapter("/content/peft-b", adapter_name="b") #이미 PEFT 모델 (model:gemma+a) 에 새로운 LoRA 어댑터를 추가로 로드. 이떄 기존에 활성화된 a는 그대로 남아있다.
# 이 시점에서 model은 두 개의 어댑터 ("a" 와 "b") 를 모두 로드하고 있지만, 단순 load_adapter로 올린 b의 경우는 활성화가 되지 않은 상태. set_adapter를 해야 가능.
model.add_weighted_adapter( # 이 함수가 어댑터 "a"나 "b"를 변경하거나 제거하는 것이 아니라, 단순히 이들을 읽어서 새로운 어댑터 "c"를 만드는 것입니다. 어댑터 "a"와 "b", 그리고 새로 만들어진 "c"는 model 객체 내에 로드된 상태로 존재합니다.
    adapters=["a", "b"],
    weights=[0.7, 0.3], #a를 70퍼, b를 30퍼로 보간한 새 어댑터(c라 지칭하자) 생성
    adapter_name=weighted_adapter_name,
    combination_type="linear",
)
model.set_adapter(weighted_adapter_name) #기존 gemma엔 c가 추가되어진다. set_adapter를 호출하면 이전에 활성화되어 있던 어댑터(들. 여기선 a)은 모두 비활성화되고 인수로 준 c만 모델에서 활성화 된다.

### Load adapters

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained("nlpai-lab/ko-gemma-2b-v1")
model = PeftModel.from_pretrained(
    base_model,
    "/content/peft-a"
)

# 다른 어댑터 로드
model.load_adapter("/content/peft-b", adapter_name="b")

# 활성 어댑터 가중치 설정
model.set_adapter("b")

In [ ]:
# 어댑터 언로드
model.unload()

# 어댑터 삭제
model.delete_adapter("b")

### LoRA 모델 추론

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel, PeftConfig

peft_model_id = "/content/drive/MyDrive/HFbookPractice/trained_model/25_02_19/peft"
config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)
#model = model.to("cuda")
model.eval()

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
inputs = tokenizer.apply_chat_template(
    [
        {"role": "user", "content": "지구상 가장 컸던 공룡은 뭐야?"},
    ],
    add_generation_prompt=True,
    return_tensors="pt",
)
gen_cfg = GenerationConfig(
    max_new_tokens=2048,
    do_sample=True,
    top_k=50,
    top_p=0.92,
    return_full_text=False,
)
with torch.no_grad():
    #outputs = model.generate(input_ids=inputs.to("cuda"), generation_config=gen_cfg)
    outputs = model.generate(input_ids=inputs, generation_config=gen_cfg)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

user
지구상 가장 컸던 공룡은 뭐야?
model
지구상에서 가장 규모가 큰 공룡은 아쿠아노돈입니다. 이 공룡은 대형 갑각류로 알려져 있으며, 몸길이가 최대 80피트까지 자랄 수 있었어요. 아쿠아노돈은 어느 시대에도 5억 년 이상 살았으며, 물고기, 갑각류, 초식동물 등 여러 종류의 생물과 함께 살았을 가능성이 있습니다.


In [5]:
inputs = tokenizer.apply_chat_template(
    [
        {"role": "user", "content": "'진실은 언제나 하나'라는 대사는 어디에서 나온거야?"},
    ],
    add_generation_prompt=True,
    return_tensors="pt",
)
gen_cfg = GenerationConfig(
    max_new_tokens=2048,
    do_sample=True,
    top_k=50,
    top_p=0.92,
    return_full_text=False,
)
with torch.no_grad():
    #outputs = model.generate(input_ids=inputs.to("cuda"), generation_config=gen_cfg)
    outputs = model.generate(input_ids=inputs, generation_config=gen_cfg)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0])

user
'진실은 언제나 하나'라는 대사는 어디에서 나온거야?
model
이 명언은 여러 영화와 TV 프로그램에 등장하였습니다. 가장 유명한 시리즈는 "아이언맨"(Iron Man)입니다. 이 명언은 먼 훗날 획득한 무기가 최근 개발된 무기를 방해하는 장치를 사용하여 극복할 수 있다는 의미입니다.
